In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
full_data_w = [] #white wine data
full_data_r = [] #red whine data

def Pulldataw(file):               #Pulling white wine data
    data_file = open(file, 'r')
    lines = data_file.readlines()
    for k in lines:
        stuff = np.array(k.split(';')) 
        full_data_w.append([stuff])    
    data_file.close()
Pulldataw('winequality-white.csv')

def Pulldatar(file):                #Pulling red wine data
    data_file = open(file, 'r')
    lines = data_file.readlines()
    for k in lines:
        stuff = np.array(k.split(';')) 
        full_data_r.append([stuff])    
    data_file.close()
Pulldatar('winequality-red.csv')

#print(full_data_r)
#print(full_data_w)

In [3]:
#splitting data into train,validate and test data (60,20,20)%
WLabels = full_data_w[0]
del full_data_w[0]
RLabels = full_data_r[0]
del full_data_r[0]

WTrain = full_data_w[:round(len(full_data_w)*0.6)]
WValid = full_data_w[round(len(full_data_w)*0.6):round(len(full_data_w)*0.8)]
WTest  = full_data_w[round(len(full_data_w)*0.8):]

RTrain = full_data_r[:round(len(full_data_r)*0.6)]
RValid = full_data_r[round(len(full_data_r)*0.6):round(len(full_data_r)*0.8)]
RTest  = full_data_r[round(len(full_data_r)*0.8):]


print(len(WTrain))
print(len(WValid))
print(len(WTest))
print()
print(len(RTrain))
print(len(RValid))
print(len(RTest))

2939
979
980

959
320
320


In [4]:
#Creating Design matrix : (DM) From Train Data


DMW = np.ones((len(WTrain),len(WTrain[0][0])+1))
DMR = np.ones((len(RTrain),len(RTrain[0][0])+1))


print(DMW.shape)
print(DMR.shape)

def CreateDM(DMtemp):  #Function to create Design Matrix
    for j in range(DMtemp.shape[0]):
        for k in range(DMtemp.shape[1]-1):
            DMtemp[j][k+1] = WTrain[j][0][k].rstrip()
    return(DMtemp)   
        
DMW = CreateDM(DMW) #Realocation of each designmatrix from ones to data
DMR = CreateDM(DMR) #Realocation of each designmatrix from ones to data





#Creating Design Matrix for validation Data
DMWValid = np.ones((len(WValid),len(WValid[0][0])+1))
DMRValid = np.ones((len(RValid),len(RValid[0][0])+1))


print(DMWValid.shape)
print(DMRValid.shape)

def CreateDM(DMtemp):  #Function to create Design Matrix
    for j in range(DMtemp.shape[0]):
        for k in range(DMtemp.shape[1]-1):
            DMtemp[j][k+1] = WValid[j][0][k].rstrip()
    return(DMtemp)   
        
DMWValid = CreateDM(DMWValid) #Realocation of each designmatrix from ones to data
DMRValid = CreateDM(DMRValid) #Realocation of each designmatrix from ones to data


  


#Create design matrix for Testing Data
DMWTest = np.ones((len(WTest),len(WTest[0][0])+1))
DMRTest = np.ones((len(RTest),len(RTest[0][0])+1))


print(DMWTest.shape)
print(DMRTest.shape)

def CreateDM(DMtemp):  #Function to create Design Matrix
    for j in range(DMtemp.shape[0]):
        for k in range(DMtemp.shape[1]-1):
            DMtemp[j][k+1] = WTest[j][0][k].rstrip()
    return(DMtemp)   
        
DMWTest = CreateDM(DMWTest) #Realocation of each designmatrix from ones to data
DMRTest = CreateDM(DMRTest) #Realocation of each designmatrix from ones to data

 

(2939, 13)
(959, 13)
(979, 13)
(320, 13)
(980, 13)
(320, 13)


In [5]:
def hFunction(DM,theta_values):
    return 1/(1+np.exp(-np.dot(DM,theta_values)))#H(x,Theta) Calculation

def GradientDescent(theta_values,theta_valuesOld,DM,alpha):#Gradient Descent Function
    while np.sqrt(np.sum(np.power(theta_values - theta_valuesOld, 2))) > 0.0005: # while euclidean norm > 0.0005 (so ϵ = 0.0005) 
        theta_valuesOld = theta_values # set old parameter values to parameter values before they are updated
        for i in range(DM.shape[0]):
            theta_values = theta_values - alpha*( (hFunction(DM[i,:12], theta_values) - DM[i][12]) * DM[i,:12] ) # update the parameters using the update rule

             
            
    return theta_values #return our theta values for this model

def convertMatrix(value,DM):  #Function to convert our design matrix for oneVsrest multiclass classification
    convDM = DM.copy()
    trueIndices=np.where(convDM[:,12]==value) #finds indices where target variable value is positive
    falseIndices=np.where(convDM[:,12]!=value) #finds indices where target variable value is negative
    convDM[trueIndices,12]=1 #sets positive values to 1
    convDM[falseIndices,12]=0 #sets negative values to 0
    return convDM

In [6]:
convertedDMR=convertMatrix(4,DMR)#create a converted design matrix where target variable only has two values 1 and 0 (1 represents the value you use as a parameter and 0 represents every other value besides the parameter value)
print(convertedDMR)
print()
print(DMR)
theta_values=np.random.uniform(size=12)#initialize our initial thetas
theta_valuesOld=np.zeros(12)
alpha=1e-4#learning rate
theta_values=GradientDescent(theta_values,theta_valuesOld,convertedDMR,alpha)
print(theta_values)#print our theta values for the model

[[ 1.    7.    0.27 ...  0.45  8.8   0.  ]
 [ 1.    6.3   0.3  ...  0.49  9.5   0.  ]
 [ 1.    8.1   0.28 ...  0.44 10.1   0.  ]
 ...
 [ 1.    7.5   0.32 ...  0.38  9.5   0.  ]
 [ 1.    6.6   0.23 ...  0.54 10.2   0.  ]
 [ 1.    6.6   0.2  ...  0.54 10.2   0.  ]]

[[ 1.    7.    0.27 ...  0.45  8.8   6.  ]
 [ 1.    6.3   0.3  ...  0.49  9.5   6.  ]
 [ 1.    8.1   0.28 ...  0.44 10.1   6.  ]
 ...
 [ 1.    7.5   0.32 ...  0.38  9.5   5.  ]
 [ 1.    6.6   0.23 ...  0.54 10.2   7.  ]
 [ 1.    6.6   0.2  ...  0.54 10.2   7.  ]]
[ 0.57859163  0.08741352  0.21437095  0.47517589 -0.06522906  0.60944534
 -0.05534131 -0.01090011  0.41023392  0.25725996  0.27502606 -0.34670094]


In [7]:
print(DMR[0])
#DMRTest = DMRTest[DMRTest[:,1].argsort()] 
value=hFunction(DMRTest[:,:12],theta_values)#our predictions for a data point in terms of probability
print(value)

[1.000e+00 7.000e+00 2.700e-01 3.600e-01 2.070e+01 4.500e-02 4.500e+01
 1.700e+02 1.001e+00 3.000e+00 4.500e-01 8.800e+00 6.000e+00]
[0.00502516 0.00883608 0.01798995 0.00230768 0.01139284 0.05765756
 0.05765756 0.01773801 0.01751686 0.02493511 0.00946956 0.00946956
 0.015281   0.00853818 0.00474151 0.0261406  0.03520115 0.00173911
 0.03165007 0.00491635 0.00944535 0.00301553 0.01019406 0.01389221
 0.02385879 0.0195184  0.00295173 0.00301553 0.00780426 0.00357161
 0.02939251 0.01019406 0.01389221 0.00510432 0.01413006 0.00846762
 0.00510432 0.01413006 0.01919021 0.03307192 0.01259942 0.01259942
 0.00214102 0.002771   0.00392119 0.02743112 0.00188572 0.02606094
 0.01238377 0.00641626 0.00675136 0.01314688 0.00514249 0.01314688
 0.06934268 0.00641626 0.00675136 0.00180307 0.00443396 0.01010035
 0.00556732 0.00544755 0.00180408 0.00056986 0.00180307 0.01290006
 0.00443396 0.00715619 0.00715619 0.00042035 0.00463216 0.00715619
 0.00042035 0.02539719 0.0305521  0.01027843 0.01027843 0.04018

In [8]:
#loop for each possible quility /10

ThetaList = []
alpha=1e-4#learning rate
valuelist = []

for k in range(11):
    convertedDMR=convertMatrix(k,DMR)
    theta_values=np.random.uniform(size=12)#initialize our initial thetas
    theta_valuesOld=np.zeros(12)
    theta_values=GradientDescent(theta_values,theta_valuesOld,convertedDMR,alpha)
    ThetaList.append(theta_values)

#loop for h functions
for t in ThetaList:
    val = hFunction(DMRTest[:,:12],t)
    valuelist.append(val)

In [9]:
print(ThetaList)
print(valuelist)

[array([ 0.94455267,  0.29809834,  0.87589833,  0.34505512,  0.42201164,
        0.5239315 ,  0.06634189, -0.49127649,  0.76196364,  0.87634668,
        0.49899452, -0.13620149]), array([ 0.97968776, -0.06354903,  0.09921868,  0.25991496,  0.53474543,
        0.44504314,  0.4662801 , -0.50782881,  0.91808156,  0.52951153,
        0.24082176, -0.00476321]), array([ 0.49992262,  0.71219353,  0.15843828,  0.04994961,  0.08990069,
        0.89991905,  0.09620431, -0.60153621,  0.95207363,  0.08293172,
        0.65988771,  0.16937994]), array([ 0.25160932, -0.02889728,  0.6615955 ,  0.69024404,  0.14359668,
        0.41278024, -0.10544708, -0.02782991,  0.55696977, -0.06858468,
        0.77205185, -0.23027176]), array([ 0.42985274,  0.10480874,  0.39778599,  0.0821967 , -0.06226975,
        0.70760032, -0.05429075, -0.01234699,  0.27879076,  0.46660163,
        0.89072536, -0.40616977]), array([ 2.49059905, -0.04025445,  3.68240885,  0.3288067 ,  0.07956952,
        0.35900293, -0.01361017,

# 